In [1]:
#imports 
import pandas as pd
import numpy as np
import os
import pickle as pkl
import datetime as datetime
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as sm
import dgl.function as fn
from tqdm import tqdm

#imports for graph creation
import torch
from sklearn.preprocessing import StandardScaler
from itertools import combinations
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

#imports for graph learning
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn
from tqdm import trange
import torch
import torch_geometric.datasets as datasets
import torch_geometric.data as data
import torch_geometric.transforms as transforms

In [2]:
import pandas as pd
import requests

def get_sitc_codes():
    # URL of the JSON file
    url = 'https://comtradeapi.un.org/files/v1/app/reference/S4.json'

    try:
        # Send a GET request to the URL and fetch the data
        response = requests.get(url)
        response.raise_for_status()  # Check that the request was successful
        
        # Load the JSON data
        data = response.json()

        # Since the JSON data might be nested, use json_normalize with appropriate arguments
        if isinstance(data, list):
            # If the top level is a list
            df = pd.json_normalize(data)
        else:
            # If the top level is a dictionary
            # Identify the key that holds the main data (adjust the path as necessary)
            main_data_key = 'results'  # Adjust this based on the actual structure
            df = pd.json_normalize(data[main_data_key])

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
    except KeyError as e:
        print(f"Error processing JSON structure: {e}")

    return df

In [3]:
class TradeNetwork:
    """
    We define a class which computes the MST trade network for a given year 
    """
    
    def __init__(self, year = 1962, data_dir = "data"):
        self.year = year
        self.data_dir = data_dir
        
    def prepare_features(self, filter_gdp = True):
        
        ###IMPORT GDP###
        #prepare GDP as a set of features 
        with open('data/all_wb_indicators.pickle', 'rb') as handle:
            features_dict = pkl.load(handle)

        self.gdp = features_dict['NY.GDP.MKTP.CD']
        scaler = StandardScaler()
        self.gdp[["prev_gdp"]] = scaler.fit_transform(np.log(self.gdp[['YR'+str(self.year-1)]]))
        self.gdp[["current_gdp"]] = scaler.fit_transform(np.log(self.gdp[['YR'+str(self.year)]]))
        #rename and keep relevant columns
        self.gdp["country_code"] = self.gdp["economy"]
        self.gdp = self.gdp[["country_code", "prev_gdp", "current_gdp"]].dropna()
        
        ###IMPORT GDP GROWTH###
        #prepare GDP growth
        self.gdp_growth = features_dict['NY.GDP.MKTP.KD.ZG']
        self.gdp_growth["prev_gdp_growth"] = self.gdp_growth['YR'+str(self.year-1)]
        self.gdp_growth["current_gdp_growth"] = self.gdp_growth['YR'+str(self.year)] 
        self.gdp_growth["future_gdp_growth"] = self.gdp_growth['YR'+str(self.year+1)]
        #rename and keep relevant columns
        self.gdp_growth["country_code"] = self.gdp_growth["economy"]
        self.gdp_growth = self.gdp_growth[["country_code", "prev_gdp_growth",
                                "current_gdp_growth", "future_gdp_growth"]].dropna()
        
        ###IMPORT GDP PER CAPITA###
        self.gdp_per_capita = features_dict['NY.GDP.PCAP.CD']
        self.gdp_per_capita["prev_gdp_per_cap"] = self.gdp_per_capita['YR'+str(self.year-1)]
        self.gdp_per_capita["current_gdp_per_cap"] = self.gdp_per_capita['YR'+str(self.year)]
        self.gdp_per_capita["future_gdp_per_cap"] = self.gdp_per_capita['YR'+str(self.year+1)]
        #rename and keep relevant columns
        self.gdp_per_capita["country_code"] = self.gdp_per_capita["economy"]
        self.gdp_per_capita = self.gdp_per_capita[["country_code", "prev_gdp_per_cap",
                                "current_gdp_per_cap", "future_gdp_per_cap"]].dropna()
        
        ###IMPORT GDP PER CAPITA GROWTH###
        self.gdp_per_capita_growth = features_dict['NY.GDP.PCAP.KD.ZG']
        self.gdp_per_capita_growth["prev_gdp_per_cap_growth"] = self.gdp_per_capita_growth['YR'+str(self.year-1)]
        self.gdp_per_capita_growth["current_gdp_per_cap_growth"] = self.gdp_per_capita_growth['YR'+str(self.year)]
        self.gdp_per_capita_growth["future_gdp_per_cap_growth"] = self.gdp_per_capita_growth['YR'+str(self.year+1)]
        
        #rename and keep relevant columns
        self.gdp_per_capita_growth["country_code"] = self.gdp_per_capita_growth["economy"]
        self.gdp_per_capita_growth = self.gdp_per_capita_growth[["country_code", "prev_gdp_per_cap_growth",
                                "current_gdp_per_cap_growth", "future_gdp_per_cap_growth"]].dropna()
        
        ###MERGE ALL DATA FEATURES###
        self.features = pd.merge(self.gdp_growth, self.gdp, on = "country_code").dropna()
        self.features = pd.merge(self.features, self.gdp_per_capita, on = "country_code").dropna()
        self.features = pd.merge(self.features, self.gdp_per_capita_growth, on = "country_code").dropna()

    def prepare_network(self):
        """
        We create an initial, import-centric trade link pandas dataframe for a given year
        """
        #get product codes
        data_dict = get_sitc_codes()
        data_cross = []
        i = 0
        for item_def in list(data_dict["text"]):
            if(i >= 2):
                data_cross.append(item_def.split(" - ", 1))
            i = i+1

        self.product_codes = pd.DataFrame(data_cross, columns = ['code', 'product'])
        self.product_codes["sitc_product_code"] = self.product_codes["code"]
        
        #get country codes
        self.country_codes = pd.read_excel("data/ISO3166.xlsx")
        self.country_codes["location_code"] = self.country_codes["Alpha-3 code"]
        self.country_codes["partner_code"] = self.country_codes["Alpha-3 code"]
        self.country_codes["country_i"] = self.country_codes["English short name"]
        self.country_codes["country_j"] = self.country_codes["English short name"]
        
        #get trade data for a given year
        trade_data = pd.read_stata(self.data_dir + "/country_partner_sitcproduct4digit_year_"+ str(self.year)+".dta") 
        #merge with product / country descriptions
        trade_data = pd.merge(trade_data, self.country_codes[["location_code", "country_i"]],on = ["location_code"])
        trade_data = pd.merge(trade_data, self.country_codes[["partner_code", "country_j"]],on = ["partner_code"])
        trade_data = pd.merge(trade_data, self.product_codes[["sitc_product_code", "product"]], 
                              on = ["sitc_product_code"])
        ###select level of product aggregation
        trade_data["product_category"] = trade_data["sitc_product_code"].apply(lambda x: x[0:1])
        
        #keep only nodes that we have features for
        #trade_data = trade_data[trade_data["location_code"].isin(self.features["country_code"])]
        #trade_data = trade_data[trade_data["partner_code"].isin(self.features["country_code"])]
        
        if (len(trade_data.groupby(["location_code", "partner_code", "sitc_product_code"])["import_value"].sum().reset_index()) != len(trade_data)):
            print("import, export, product combination not unique!")
        self.trade_data1 = trade_data
        #from import-export table, create only import table
        #extract imports
        imports1 = trade_data[['location_id', 'partner_id', 'product_id', 'year',
               'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        imports1 = imports1[imports1["import_value"] != 0]
        #transform records of exports into imports
        imports2 = trade_data[['location_id', 'partner_id', 'product_id', 'year',
               'export_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        imports2["temp1"] = imports2['partner_code']
        imports2["temp2"] = imports2['location_code']

        imports2['location_code'] = imports2["temp1"]
        imports2['partner_code'] = imports2["temp2"]
        imports2["import_value"] = imports2["export_value"]
        imports2 = imports2[imports2["import_value"] != 0]
        imports2 = imports2[['location_id', 'partner_id', 'product_id', 'year',
               'import_value', 'sitc_eci', 'sitc_coi', 'location_code', 'partner_code',
               'sitc_product_code', 'country_i', 'country_j', 'product', "product_category"]]
        
        imports_table = pd.concat([imports1, imports2]).drop_duplicates()
        
        #rename columns for better clarity
        imports_table["importer_code"] = imports_table["location_code"]
        imports_table["exporter_code"] = imports_table["partner_code"]
        imports_table["importer_name"] = imports_table["country_i"]
        imports_table["exporter_name"] = imports_table["country_j"]
        
        cols = ["importer_code", "exporter_code", "importer_name", "exporter_name",
               'product_id', 'year', 'import_value', 'sitc_eci', 'sitc_coi',
               'sitc_product_code', 'product', "product_category"]
        imports_table = imports_table[cols]
        
        exporter_total = imports_table.groupby(["exporter_code"])["import_value"].sum().reset_index()
        exporter_total = exporter_total.rename(columns = {"import_value": "export_total"})
        
        importer_total = imports_table.groupby(["importer_code"])["import_value"].sum().reset_index()
        importer_total = importer_total.rename(columns = {"import_value": "import_total"})
        
        #sum imports across all products between countries into single value 
        imports_table_grouped = imports_table.groupby(["importer_code", "exporter_code"])["import_value"].sum().reset_index()
        
        #sum exports in each category 
        self.export_types = imports_table.groupby(["exporter_code", "product_category"])["import_value"].sum().reset_index()
        self.export_types = pd.merge(self.export_types, exporter_total, on = "exporter_code")
        #multiply by 100 to allow weights to scale better in GNN
        self.export_types["category_fraction"] = self.export_types.import_value/self.export_types.export_total*10
        ss = StandardScaler()
        columns = list(set(self.export_types["product_category"]))
        self.export_types = self.export_types[["exporter_code", "product_category", "category_fraction"]]\
        .pivot(index = ["exporter_code"], columns = ["product_category"], values = "category_fraction")\
        .reset_index().fillna(0)
        #rename columns
        rename_columns = []
        for col in self.export_types.columns:
            if(col == "exporter_code"):
                rename_columns.append(col)
            else:
                rename_columns.append("resource_" + col)
        self.export_types.columns = rename_columns
        self.export_types = self.export_types.rename(columns = {"exporter_code": "country_code"})
        self.features = pd.merge(self.features, self.export_types, 
                                on = "country_code", how = "left")
        
        #look at fraction of goods traded with each counterparty
        imports_table_grouped = pd.merge(imports_table_grouped, exporter_total, how = "left")
        imports_table_grouped["export_percent"] = imports_table_grouped["import_value"]/imports_table_grouped["export_total"]
        scaler = StandardScaler()
        imports_table_grouped[["export_percent_feature"]] = scaler.fit_transform(np.log(imports_table_grouped[["export_percent"]]))
        imports_table_grouped["export_percent_feature"] = imports_table_grouped["export_percent_feature"] + abs(min(imports_table_grouped["export_percent_feature"]))
        
        imports_table_grouped = pd.merge(imports_table_grouped, importer_total, how = "left")
        imports_table_grouped["import_percent"] = imports_table_grouped["import_value"]/imports_table_grouped["import_total"]
        scaler = StandardScaler()
        imports_table_grouped[["import_percent_feature"]] = scaler.fit_transform(np.log(imports_table_grouped[["import_percent"]]))
        imports_table_grouped["import_percent_feature"] = imports_table_grouped["import_percent_feature"] + abs(min(imports_table_grouped["import_percent_feature"]))
        
        self.trade_data = imports_table_grouped

    def graph_create(self, exporter = True,
            node_features = ['prev_gdp_growth', 'current_gdp_growth','prev_gdp','current_gdp'],
            node_labels = 'future_gdp_growth'):
        
        if(exporter):
            center_node = "exporter_code"
            neighbors = "importer_code"
            edge_features = 'export_percent'
        
        #filter features and nodes to ones that are connected to others in trade data
        list_active_countries = list(set(list(self.trade_data ["importer_code"])+\
                        list(self.trade_data ["exporter_code"])))
        
        
        self.features = self.features[self.features["country_code"].isin(list_active_countries)].reset_index()
        self.features.fillna(0, inplace = True)
        self.features["node_numbers"] = self.features.index
        #create lookup dictionary making node number / node features combatible with ordering of nodes
        #in our edge table

        self.node_lookup1 = self.features.set_index('node_numbers').to_dict()['country_code']
        self.node_lookup2 = self.features.set_index('country_code').to_dict()['node_numbers']
        
        #get individual country's features
        self.regression_table = pd.merge(self.features, self.trade_data,
                        left_on = "country_code",
                        right_on = center_node, how = 'right')
        #get features for trade partners
        self.regression_table = pd.merge(self.features, self.regression_table,
                                        left_on = "country_code",
                                        right_on = neighbors, how = "right",
                                        suffixes = ("_neighbors", ""))
        
        self.trade_data = self.trade_data[self.trade_data[neighbors].isin(self.node_lookup2)]
        self.trade_data = self.trade_data[self.trade_data[center_node].isin(self.node_lookup2)]

        self.regression_table["source"] = self.trade_data[neighbors].apply(lambda x: self.node_lookup2[x])
        self.regression_table["target"] = self.trade_data[center_node].apply(lambda x: self.node_lookup2[x])    

        self.regression_table = self.regression_table.dropna()
        #filter only to relevant columns
        self.relevant_columns = ["source", "target"]
        self.relevant_columns.extend(node_features)
        self.relevant_columns.append(node_labels)
        self.graph_table = self.regression_table[self.relevant_columns]
        
        if(self.graph_table.isnull().values.any()): print("edges contain null / inf values")

        self.node_attributes = torch.tensor(np.array(self.features[node_features]))\
        .to(torch.float)
        self.source_nodes = list(self.graph_table["source"])
        self.target_nodes = list(self.graph_table["target"])
        self.edge_attributes = list(self.trade_data[edge_features])

        self.pyg_graph = data.Data(x = self.node_attributes,
                                   edge_index = torch.tensor([self.source_nodes, self.target_nodes]),
                                   edge_attr = torch.tensor(self.edge_attributes).to(torch.float),
                                   y = torch.tensor(list(self.features[node_labels])).to(torch.float))

        

# Graph Setup

In [5]:
import random
years = list(range(1962,2021))

# Determine the sizes of the train, validation, and test sets
train_size = int(len(years) * 0.6)
val_size = int(len(years) * 0.20)

# Get a random subset for the train set
train_years = random.sample(years, train_size)

# Remove the years in the train set from the list of years
years = [year for year in years if year not in train_years]

# Get a random subset for the validation set
val_years = random.sample(years, val_size)

# Remove the years in the validation set from the list of years
test_years = [year for year in years if year not in val_years]

print("Train years:", train_years)
print("Validation years:", val_years)
print("Test years:", test_years)

Train years: [2005, 1969, 2002, 1997, 1993, 1982, 2001, 2000, 1962, 1985, 1978, 2016, 1986, 1987, 1989, 1971, 2013, 1996, 1995, 1967, 2017, 1974, 1990, 1977, 1980, 2014, 1965, 1984, 2006, 1973, 1968, 1981, 1970, 1991, 2019]
Validation years: [1975, 1983, 2009, 1966, 1999, 1988, 2007, 1979, 1972, 2015, 2003]
Test years: [1963, 1964, 1976, 1992, 1994, 1998, 2004, 2008, 2010, 2011, 2012, 2018, 2020]


In [7]:
train_graphs = []
val_graphs = []
test_graphs = []
i = 0

all_nodes = ['ABW', 'AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM',
       'ATG', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD',
       'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BLZ', 'BMU', 'BOL', 'BRA',
       'BRB', 'BRN', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN',
       'CIV', 'CMR', 'COD', 'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB',
       'CUW', 'CYM', 'CYP', 'CZE', 'DEU', 'DMA', 'DNK', 'DOM', 'DZA',
       'ECU', 'EGY', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'FSM',
       'GAB', 'GBR', 'GEO', 'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC',
       'GRD', 'GRL', 'GTM', 'GUM', 'GUY', 'HKG', 'HND', 'HRV', 'HTI',
       'HUN', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA',
       'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KNA', 'KOR',
       'KWT', 'LAO', 'LBN', 'LBR', 'LBY', 'LCA', 'LKA', 'LSO', 'LTU',
       'LUX', 'LVA', 'MAC', 'MAR', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL',
       'MKD', 'MLI', 'MLT', 'MMR', 'MNE', 'MNG', 'MNP', 'MOZ', 'MRT',
       'MUS', 'MWI', 'MYS', 'NAM', 'NER', 'NGA', 'NIC', 'NLD', 'NOR',
       'NPL', 'NRU', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PLW',
       'PNG', 'POL', 'PRT', 'PRY', 'PSE', 'PYF', 'QAT', 'ROU', 'RUS',
       'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLB', 'SLE', 'SLV', 'SMR',
       'SRB', 'SSD', 'STP', 'SUR', 'SVK', 'SVN', 'SWE', 'SWZ', 'SXM',
       'SYC', 'SYR', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TLS', 'TON',
       'TTO', 'TUN', 'TUR', 'TUV', 'TZA', 'UGA', 'UKR', 'URY', 'USA',
       'UZB', 'VCT', 'VEN', 'VNM', 'VUT', 'WSM', 'YEM', 'ZAF', 'ZMB',
       'ZWE']

years = list(range(1962,2021))
for year in tqdm(years):
    print(str(year), end='\r')
    
    trade = TradeNetwork(year = year)
    trade.prepare_features()
    trade.prepare_network()
    trade.graph_create(node_features = ['prev_gdp_per_cap_growth', 'current_gdp_per_cap_growth',
    'resource_0', 'resource_1', 'resource_2', 'resource_3', 'resource_4', 'resource_5', 'resource_6', 'resource_7',
       'resource_8', 'resource_9'],
        node_labels = 'future_gdp_per_cap_growth')
    
    current_nodes = list(trade.node_lookup2.keys())
    num_features = trade.pyg_graph.x.size(1)
    zeros = torch.zeros(1, num_features)

    for i in range(len(all_nodes)):
        if i >= len(current_nodes):
            current_nodes.insert(i, all_nodes[i])
            tensor_before = trade.pyg_graph.x
            new_tensor = torch.cat([tensor_before, zeros], dim=0)
            trade.pyg_graph.x = new_tensor

            node_attr_before = trade.node_attributes
            trade.node_attributes = torch.cat([node_attr_before, zeros], dim=0)

        elif(all_nodes[i]!=current_nodes[i]):
            current_nodes.insert(i, all_nodes[i])
            if i==0:
                tensor_after = trade.pyg_graph.x[i:]
                new_tensor = torch.cat([zeros, tensor_after], dim=0)
                trade.pyg_graph.x = new_tensor

                node_attr_after = trade.node_attributes[i:]
                trade.node_attributes = torch.cat([zeros, node_attr_after], dim=0)
            else:
                tensor_before = trade.pyg_graph.x[:i]
                tensor_after = trade.pyg_graph.x[i:]
                new_tensor = torch.cat([tensor_before, zeros, tensor_after], dim=0)
                trade.pyg_graph.x = new_tensor

                node_attr_before = trade.node_attributes[:i]
                node_attr_after = trade.node_attributes[i:]
                trade.node_attributes = torch.cat([node_attr_before, zeros, node_attr_after], dim=0)

    for i in range(len(current_nodes) - 1, -1, -1):
        if current_nodes[i] not in all_nodes:
            node_to_remove = current_nodes.pop(i)

            tensor_before = trade.pyg_graph.x[:i]
            tensor_after = trade.pyg_graph.x[i+1:]
            new_tensor = torch.cat([tensor_before, tensor_after], dim=0)
            trade.pyg_graph.x = new_tensor

            node_attr_before = trade.node_attributes[:i]
            node_attr_after = trade.node_attributes[i+1:]
            trade.node_attributes = torch.cat([node_attr_before, node_attr_after], dim=0)

    if len(current_nodes) > len(all_nodes):
        num_diff = len(current_nodes) - len(all_nodes)
        trade.pyg_graph.x = trade.pyg_graph.x[:-num_diff]
        trade.node_attributes = trade.node_attributes[:-num_diff]
        current_nodes = current_nodes[:-num_diff]
    
    if current_nodes != all_nodes:
        print("Nodes not equal")
        break

    if(year in val_years):
        val_graphs.append(trade.pyg_graph)
    elif(year in test_years):
        test_graphs.append(trade.pyg_graph)
    else: 
        train_graphs.append(trade.pyg_graph)
        
    trade.features["year"] = year
    
    if(i == 0):
        trade_df = trade.features
    else: 
        trade_df = pd.concat([trade_df, trade.features])
        
    i = i+1
    print(trade.node_attributes.size())

  0%|          | 0/59 [00:00<?, ?it/s]

  2%|▏         | 1/59 [00:07<06:51,  7.10s/it]

torch.Size([201, 12])


  3%|▎         | 2/59 [00:13<06:24,  6.74s/it]

torch.Size([201, 12])


...

torch.Size([201, 12])


 98%|█████████▊| 58/59 [11:39<00:19, 19.94s/it]

torch.Size([201, 12])


100%|██████████| 59/59 [11:58<00:00, 12.18s/it]

torch.Size([201, 12])


In [8]:
import pickle as pkl
with open("pygcn/train_graphs.pickle", "wb") as f:
    pkl.dump(train_graphs, f)

with open("pygcn/val_graphs.pickle", "wb") as f:
    pkl.dump(val_graphs, f)

with open("pygcn/test_graphs.pickle", "wb") as f:
    pkl.dump(test_graphs, f)

In [57]:
import pickle as pkl
with open("pygcn/train_graphs.pickle", "rb") as f:
    train_graphs = pkl.load(f)

with open("pygcn/val_graphs.pickle", "rb") as f:  
    val_graphs = pkl.load(f)

with open("pygcn/test_graphs.pickle", "rb") as f:         
    test_graphs = pkl.load(f)

In [9]:
from torch_geometric.data import DataLoader
test_loader = DataLoader(test_graphs, batch_size=32)
train_loader = DataLoader(train_graphs, batch_size=32)
val_loader = DataLoader(val_graphs, batch_size=32)

/Users/sinclaireschuetze/anaconda3/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


## sGNN with GCN Encoder and 3 Features

In [10]:
def check_crisis_years(year_pairs, crisis_years):
    result = []
    for pair in year_pairs:
        start, end = pair
        # Check if any crisis year is between the pair or equals the later year
        if any(start < year <= end for year in crisis_years):
            result.append(0)
        else:
            result.append(1)
    return result

In [11]:
crisis_years = [1983, 1982, 2008, 2002, 2016, 1967, 1962, 1989, 2012, 1963, 1993, 1986, 1996,1978]

def get_year_pairs(year_range):
    return [(year1, year2) for year1 in year_range for year2 in year_range if year2 >= year1]

def get_loader_pairs(dataset):
    return [(dataset[i], dataset[j]) for i in range(len(dataset)) for j in range(len(dataset)) if j >= i]

train_pairs = get_year_pairs(train_years)
val_pairs = get_year_pairs(val_years)

train_y = check_crisis_years(train_pairs, crisis_years)
val_y = check_crisis_years(val_pairs, crisis_years)

train_loader_pairs = get_loader_pairs(train_loader.dataset)
val_loader_pairs = get_loader_pairs(val_loader.dataset)

In [12]:
train_torch_y = torch.tensor(np.array(train_y))
val_torch_y = torch.tensor(np.array(val_y))

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv, global_sort_pool
from torch_geometric.nn.aggr import SortAggregation
from torch.nn import Linear, LayerNorm, ReLU, Sigmoid
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR


class GNN(torch.nn.Module):
    def __init__(self, num_features):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(num_features, 128)
        self.conv2 = GCNConv(128, 64)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index.to(torch.int64)
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        return x

class SiameseGNN(torch.nn.Module):
    def __init__(self, num_features):
        super(SiameseGNN, self).__init__()
        self.gnn = GNN(num_features)
        self.sort_aggr = SortAggregation(k=50)
        self.fc1 = Linear(10050, 128)  # Adjust input size according to pooling output
        self.norm1 = LayerNorm(128)
        self.relu1 = ReLU()
        self.fc2 = Linear(128, 64)
        self.norm2 = LayerNorm(64)
        self.relu2 = ReLU()
        self.fc3 = Linear(64, 1)
        self.sigmoid = Sigmoid()

    def forward(self, data1, data2):
        out1 = self.gnn(data1)
        out2 = self.gnn(data2)
        out = torch.cdist(out1, out2, p=2)
        out = self.sort_aggr(out, data1.batch)
        out = out.view(out.size(0), -1)  # Flatten the pooled output
        out = self.fc1(out)
        out = self.norm1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.norm2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

# DataLoader that loads pairs of graphs
train_loader = train_loader_pairs
val_loader = val_loader_pairs

model = SiameseGNN(num_features=train_loader[0][0].num_node_features)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Adjust step_size and gamma as needed
criterion = nn.BCELoss()

for epoch in tqdm(range(20)):
    model.train()
    train_losses = []
    for i in range(len(train_loader)):
        optimizer.zero_grad()
        out = model(train_loader[i][0], train_loader[i][1])
        loss = criterion(out.squeeze(), train_torch_y[i].to(torch.float))
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    scheduler.step()  # Add this line to update the learning rate

    # Validation
    model.eval()
    with torch.no_grad():
        val_losses = []
        correct = 0
        total = 0
        for i in range(len(val_loader)):
            out = model(val_loader[i][0], val_loader[i][1])
            val_loss = criterion(out.squeeze(), val_torch_y[i].to(torch.float))
            val_losses.append(val_loss.item())

            predictions = torch.round(out.squeeze())
            correct += (predictions == val_torch_y[i]).sum().item()
            total += 1

        val_loss = sum(val_losses) / len(val_losses)
        val_accuracy = correct / total

    print(f'Epoch: {epoch+1}, Training Loss: {sum(train_losses)/len(train_losses)}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')


  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:12<03:54, 12.37s/it]

Epoch: 1, Training Loss: 0.405819375576481, Validation Loss: 0.5019316388802095, Validation Accuracy: 0.803030303030303


 10%|█         | 2/20 [00:26<03:56, 13.13s/it]

Epoch: 2, Training Loss: 0.39730588025043884, Validation Loss: 0.5016955918434894, Validation Accuracy: 0.803030303030303


...

Epoch: 18, Training Loss: 0.38852022199167147, Validation Loss: 0.5273394104883526, Validation Accuracy: 0.7878787878787878


 95%|█████████▌| 19/20 [03:53<00:12, 12.12s/it]

Epoch: 19, Training Loss: 0.38830634000755493, Validation Loss: 0.5275801982391964, Validation Accuracy: 0.7878787878787878


100%|██████████| 20/20 [04:04<00:00, 12.23s/it]

Epoch: 20, Training Loss: 0.3881064667706452, Validation Loss: 0.5276957211846655, Validation Accuracy: 0.7878787878787878
